In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().set("spark.ui.port", "4050")

# creamos el contexto y la sesión
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/28 03:17:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [55]:
data_covid = sc.textFile('./data/covid_articles_raw.csv')

In [56]:
type(data_covid)

pyspark.rdd.RDD

In [57]:
data_covid.take(5)

['title,content,category',
 'Looking into the truth about modern workplace environments,"Hi, what are you looking for?',
 'By',
 'Published',
 'Workplaces are being transformed, according to the Olivet Nazarene University study ( “ The Truth about Modern Offices “). The days of men and women fitted in formal clothing and working under the harsh glare fluorescent lighting are slipping away. The trends of today are reflective of casual dress codes and soft furnishings.']

### Creamos una función que "limpia" cada línea: se remueven signos de puntuación y otros caracteres. Adicionalmente convertimos el texto a minúscula

In [58]:
def clean_text(line:str) -> str:
    punct = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
    words = ['covid19','said','2020','2021','coronavirus','pandemic']
    lowercased = line.lower()
    for word in words:
        lowercased = lowercased.replace(word.strip(), '')
    return lowercased.translate(str.maketrans('','',punct)).strip()

In [59]:
data_covid = data_covid.map(clean_text)

In [60]:
data_covid.take(10)

['titlecontentcategory',
 'looking into the truth about modern workplace environmentshi what are you looking for',
 'by',
 'published',
 'workplaces are being transformed according to the olivet nazarene university study  “ the truth about modern offices “ the days of men and women fitted in formal clothing and working under the harsh glare fluorescent lighting are slipping away the trends of today are reflective of casual dress codes and soft furnishings',
 'there are signs too that the ‘ animal penning’  read cubicles are starting to be withdrawn and replaced with open floor plans  indications are that over half of workplaces – 51 percent – have gravitated towards the ‘ open office’ or open floor plan concept',
 'the survey also showed that 77 percent of us located workers are reportedly happy with the way their office is set up however as an indication of thoughts about openplan areas those with private offices stated they were happiest',
 'through changing technology more employees

In [61]:
all_words = data_covid.flatMap(lambda line: line.split())

In [62]:
all_words.take(20)

['titlecontentcategory',
 'looking',
 'into',
 'the',
 'truth',
 'about',
 'modern',
 'workplace',
 'environmentshi',
 'what',
 'are',
 'you',
 'looking',
 'for',
 'by',
 'published',
 'workplaces',
 'are',
 'being',
 'transformed']

In [63]:
data_covid_count = all_words.map(lambda word: (word,1))

In [64]:
data_covid_rbk = data_covid_count.reduceByKey(lambda a,b: a+b).sortByKey()

In [ ]:
data_covid_rbk.persist()

In [ ]:
data_covid_rbk.take(10)

### Truco para encontrar las palabras con mas ocurrencias:

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords

### Dejamos solo las palabras que no sean stopwords

In [ ]:
data_covid_rbk = data_covid_rbk.filter(lambda tupla: tupla[0] not in stopwords)

In [ ]:
data_covid_rbk.map(lambda tupla: (tupla[1],tupla[0])).sortByKey(False).take(20)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
word_dict = {tupla[0]:float(tupla[1]) for tupla in data_covid_rbk.collect()}

In [ ]:
wc = WordCloud(width=3000,height=1000).generate_from_frequencies(word_dict)

In [ ]:
plt.figure(figsize=(30, 10))
plt.imshow(wc)
plt.axis("off")